In [1]:
# preliminary
from tensorflow.keras.datasets import imdb
from tensorflow import keras
from tensorflow.keras import layers 
import numpy as np
import tensorflow as tf

In [2]:
# load imdb data
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

In [3]:
# 
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

# define model
def build_model():
    model = keras.Sequential([
        layers.Dense(4, activation="relu"),
        layers.Dense(4, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])
    return model

In [4]:
print(train_labels)

[1 0 0 ... 0 1 0]


In [5]:
class RootMeanSquaredError(keras.metrics.Metric):
    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32"
        )
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)
    
    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))
    
    def reset_state(self):
        self.mse_sum.assign(0.)
        self.total_samples.assign(0)

In [6]:
class F1Score(keras.metrics.Metric):
    def __init__(self, name="F1score_User", **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="true_positives", initializer="zeros")
        self.predicted_positives = self.add_weight(name="predicted_positives", initializer="zeros") 
        self.actual_positives = self.add_weight(name="actual_positives", initializer="zeros")
            
    def update_state(self, y_true, y_pred, sample_weight=None):
        # For binary classification, threshold predictions at 0.5
        y_pred_classes = tf.cast(y_pred > 0.5, "float32")
        y_true = tf.cast(y_true, "float32")

        actual_positives = tf.reduce_sum(y_true)
        true_positives = tf.reduce_sum(y_true * y_pred_classes)
        predicted_positives = tf.reduce_sum(y_pred_classes)
        
        self.true_positives.assign_add(true_positives)
        self.predicted_positives.assign_add(predicted_positives)     
        self.actual_positives.assign_add(predicted_positives)     
    
    def result(self):
        precision = ( tf.cast(self.true_positives, tf.float32) 
              / tf.cast(self.actual_positives, tf.float32) + tf.keras.backend.epsilon())
        recall = ( tf.cast(self.true_positives, tf.float32) 
              / tf.cast(self.actual_positives, tf.float32) + tf.keras.backend.epsilon())
        return  (2 * recall * precision) / (recall + precision + tf.keras.backend.epsilon())
    
    def reset_state(self):
        self.true_positives.assign(0)
        self.predicted_positives.assign(0)
        self.actual_positives.assign(0)
                

In [7]:
# define model
model_new = build_model()

In [8]:
model_new.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                 metrics=["accuracy", F1Score()])

In [ ]:
history_new = model_new.fit(
    train_data, train_labels,
    epochs=20, batch_size=4, validation_split=0.4
)

Epoch 1/20
3750/3750 [==============================] - 13s 3ms/step - loss: 0.3641 - accuracy: 0.8499 - F1score_User: 0.8537 - val_loss: 0.2911 - val_accuracy: 0.8903 - val_F1score_User: 0.8652
Epoch 2/20
3750/3750 [==============================] - 9s 2ms/step - loss: 0.2419 - accuracy: 0.9113 - F1score_User: 0.9068 - val_loss: 0.2985 - val_accuracy: 0.8929 - val_F1score_User: 0.8691
Epoch 3/20
3750/3750 [==============================] - 9s 2ms/step - loss: 0.2149 - accuracy: 0.9219 - F1score_User: 0.9179 - val_loss: 0.3052 - val_accuracy: 0.8931 - val_F1score_User: 0.8876
Epoch 4/20
3750/3750 [==============================] - 9s 2ms/step - loss: 0.1977 - accuracy: 0.9283 - F1score_User: 0.9234 - val_loss: 0.3156 - val_accuracy: 0.8905 - val_F1score_User: 0.8793
Epoch 5/20
3750/3750 [==============================] - 9s 2ms/step - loss: 0.1897 - accuracy: 0.9335 - F1score_User: 0.9283 - val_loss: 0.3224 - val_accuracy: 0.8908 - val_F1score_User: 0.8850
Epoch 6/20
3750/3750 [=======

In [ ]:
a = tf.constant([[0.5],[1],[1]], dtype="float32")


In [ ]:
s = tf.reduce_sum(a)

In [ ]:
s